In [1]:
from pyspark.sql import SparkSession

In [2]:
STS_RAW_TRAIN_PATH = '../data/sts/training.1600000.processed.noemoticon.csv'
STS_RAW_TEST_PATH  = '../data/ststestdata.manual.2009.06.14.csv'

STS_PROCESED_TRAIN_PATH = '../data/processed/sts/sts_train'
STS_PROCESED_TEST_PATH  = '../data/processed/sts/sts_test'

COVID_PROCESSED_PATH = '../data/proccesed/full-tweets-sanitized/tweets-santized'

# Load Data

In [ ]:
df_